# Function Calling with OpenAI
The notebook includes a mock database of books and functions to list, search, and retrieve books. It illustrates how to use the AI model to interpret a user's question, determine the appropriate function to call, and execute the function. This notebook serves as a practical guide for leveraging AIConfig and function calling with OpenAI models. Read more about [Function Calling with Open AI](https://openai.com/blog/function-calling-and-other-api-updates) and [AIConfig for prompt and model management](https://github.com/lastmile-ai/aiconfig).

In [ ]:
# Install and Set OpenAI Key
!pip install python-aiconfig
from google.colab import userdata

import openai
import os

openai.api_key = userdata.get('openai_key')

In [33]:
# Load AI Config
from aiconfig.Config import AIConfigRuntime, CallbackManager, InferenceOptions

config_file_path = "function-call.aiconfig.json"
config = AIConfigRuntime.load(config_file_path)
config.callback_manager = CallbackManager([])

In [ ]:
# Define db of books
db = [
    {
        'id': 'a1',
        'name': 'To Kill a Mockingbird',
        'genre': 'historical',
        'description': ('Compassionate, dramatic, and deeply moving, "To Kill A Mockingbird" takes readers to the roots of human behavior - to innocence and experience, kindness and cruelty, love and hatred, humor and pathos. Now with over 18 million copies in print and translated into forty languages, this regional story by a young Alabama woman claims universal appeal. Harper Lee always considered her book to be a simple love story. Today it is regarded as a masterpiece of American literature.'),
    },
    {
        'id': 'a2',
        'name': 'All the Light We Cannot See',
        'genre': 'historical',
        'description': ('In a mining town in Germany, Werner Pfennig, an orphan, grows up with his younger sister, enchanted by a crude radio they find that brings them news and stories from places they have never seen or imagined. Werner becomes an expert at building and fixing these crucial new instruments and is enlisted to use his talent to track down the resistance. Deftly interweaving the lives of Marie-Laure and Werner, Doerr illuminates the ways, against all odds, people try to be good to one another.'),
    },
    {
        'id': 'a3',
        'name': 'Where the Crawdads Sing',
        'genre': 'historical',
        'description': ('For years, rumors of the “Marsh Girl” haunted Barkley Cove, a quiet fishing village. Kya Clark is barefoot and wild; unfit for polite society. So in late 1969, when the popular Chase Andrews is found dead, locals immediately suspect her.\n\n'
                        'But Kya is not what they say. A born naturalist with just one day of school, she takes life\'s lessons from the land, learning the real ways of the world from the dishonest signals of fireflies. But while she has the skills to live in solitude forever, the time comes when she yearns to be touched and loved. Drawn to two young men from town, who are each intrigued by her wild beauty, Kya opens herself to a new and startling world—until the unthinkable happens.'),
    },
]

In [36]:
# Define the functions: list, search, get

# The 'list' function returns a list of books in a specified genre.
def list(genre):
    return [item for item in db if item['genre'] == genre]

# The 'search' function returns a list of books that match the provided name.
def search(name):
    return [item for item in db if name in item['name']]

# The 'get' function returns detailed information about a book based on its ID.
# Note: This function accepts only IDs, not names. Use the 'search' function to find a book's ID.
def get(id):
    for item in db:
        if item['id'] == id:
            return item
    return None

In [ ]:
# 'call_function' is a helper function that executes the function specified by the LLM's output for 'function_call'.
# It handles 'list', 'search', or 'get' functions, and raises a ValueError for unknown functions.
import json

def call_function(function_call):
    args = json.loads(function_call['arguments'])
    name = function_call['name']

    if name == 'list':
        return list(args['genre'])
    elif name == 'search':
        return search(args['name'])
    elif name == 'get':
        return get(args['id'])
    else:
        raise ValueError('No function found')

In [34]:
# Run recommendBook prompt with gpt-3.5 and determine right function to call based on user question
params = {"book":"Where the Crawdads Sing"}
inference_options = InferenceOptions(stream=True)

completion = await config.run("recommendBook", params, options=inference_options)

func_call = completion[0].data.get('function_call')
print(func_call)


{'arguments': '{\n  "name": "Where the Crawdads Sing"\n}', 'name': 'search'}


In [35]:
# Run call_function to execute the LLM-specified function (list, search, get)
value = call_function(func_call)
print(json.dumps(value, indent=4)
)

[
    {
        "id": "a3",
        "name": "Where the Crawdads Sing",
        "genre": "historical",
        "description": "For years, rumors of the \u201cMarsh Girl\u201d haunted Barkley Cove, a quiet fishing village. Kya Clark is barefoot and wild; unfit for polite society. So in late 1969, when the popular Chase Andrews is found dead, locals immediately suspect her.\n\nBut Kya is not what they say. A born naturalist with just one day of school, she takes life's lessons from the land, learning the real ways of the world from the dishonest signals of fireflies. But while she has the skills to live in solitude forever, the time comes when she yearns to be touched and loved. Drawn to two young men from town, who are each intrigued by her wild beauty, Kya opens herself to a new and startling world\u2014until the unthinkable happens."
    }
]
